<a href="https://colab.research.google.com/github/BoringName1234/Assignment-1/blob/main/BB84_Protocol.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Start by installing qiskit and checking the install

In [ ]:
!pip install qiskit qiskit-aer pylatexenc

In [ ]:
import qiskit
print("Qiskit version:", qiskit.__version__)

Qiskit version: 2.2.3


First, we should define a method to create the random key

In [ ]:
def Generate_Key(num_bits):
  key = []
  for i in range (num_bits):
        # Decide if the bit will be zero or one
        key.append(random.randint(0,1))

  return key

We also need a function that creates the gates used to alter the phase for transmission of reception.

In [ ]:
def Gate_Choices(num_bits):
  gates = []
  for i in range (num_bits):
    gates.append(random.randint(0,1))

  return gates

We can now define the circut that represents the quantum transmissions

In [ ]:
from qiskit import QuantumCircuit

def BB84_Circuit(key, alice_gates, bob_gates, eve_gates = []):
  num_qubits = len(key)
  qc = QuantumCircuit(num_qubits)

  for i, bit in enumerate(key):
    # If the bit is 1, apply NOT gate
    if bit == 1:
      qc.x(i)

    # Then apply the transmission gate
    if alice_gates[i] == 1:
      qc.h(i)

  # Configure the eavesdropper - default has no eve
  for i in range (len(eve_gates)):
    if eve_gates[i] == 1:
      qc.h(i)

  # Add Bob's measurement gates
  for i in range (len(bob_gates)):
    if(bob_gates[i] == 1):
      qc.h(i)
  qc.measure_all()

  return qc

We need a custom implementation of the Most Frequent Count function, that doesn't crash when there are multiple equally frequent counts


In [ ]:
def most_frequent(counts):
  """Return the most frequent count

  Returns:
      str: The bit string for the most frequent result. If there are multiple most frequent results, return the first.
  """

  max_value = max(counts.values())
  max_values_counts = [x[0] for x in counts.items() if x[1] == max_value]

  return max_values_counts[0]

This code can define an agent in the protocol

In [ ]:
class Agent:
  def __init__(self, num_bits):
    self.

Now we can define the simulation

In [ ]:
import random
import numpy as np
from qiskit import QuantumCircuit
from qiskit_aer import Aer, AerSimulator
from qiskit import transpile

def BB84_Simulation(num_bits, noise=False, eavesdropper=False):

  # Start by generating a random key
  key = Generate_Key(num_bits)
  # Alice and Bob need to choose their transmission gates
  alice_gates = Gate_Choices(num_bits)
  bob_gates = Gate_Choices(num_bits)

  # Define the circuit, then get the results
  qc = BB84_Circuit(key, alice_gates, bob_gates)

  if(noise == False):
    noise_level = 0
    #backend = Aer.get_backend('qasm_simulator')
  #else:
    #TODO
    #print("Unimplemented")

  backend = Aer.get_backend('qasm_simulator')
  new_cirq = transpile(qc, backend)
  job = backend.run(new_cirq)
  counts = job.result().get_counts()

  # Retrieve the most common result of the circut
  measured_bits = most_frequent(counts)
  # Then reverse the bitsring, as the circut measures backwards
  measured_bits = measured_bits[::-1]
  bob_results = [int(x) for x in str(measured_bits)]

  # Reciever announces their bases, and sender compares
  # This is the sifting stage
  alice_sifted_key = []
  bob_sifted_key = []
  num_correct = 0
  for i in range (num_bits):
    if alice_gates[i] == bob_gates[i]:
      alice_sifted_key.append(key[i])
      bob_sifted_key.append(bob_results[i])

  print("Alice sifted key: ", alice_sifted_key)
  print("Bob sifted key: ", bob_sifted_key)

  #Announce the first 50% of bits with the correct base
  announcement_num = len(alice_sifted_key)//2
  print(announcement_num)
  alice_announcement = alice_sifted_key[:announcement_num]
  bob_announcement = bob_sifted_key[:announcement_num]
  count = 0
  for i in range (announcement_num):
    if alice_announcement[i] == bob_announcement[i]:
      count += 1

  success_rate = count/announcement_num
  print("Success rate: ", success_rate)
  if(success_rate < 1 and noise == False):
    print("Eavesdropper detected,  abandoning key distribution.")

  elif(success_rate >= 1-noise_level):
    print("No eavesdropper detected.")
    alice_key = alice_sifted_key[announcement_num:]
    bob_key = bob_sifted_key[announcement_num:]
    print("Alice key: ", alice_key)
    print("Bob key: ", bob_key)

  else:
    print("Eavesdropper detected, abandoning key distribution.")



In [ ]:
BB84_Simulation(20)

11001110010001100111
[1, 1, 0, 0, 1, 1, 1, 0, 0, 1, 0, 0, 0, 1, 1, 0, 0, 1, 1, 1]
[0, 1, 0, 0, 1, 1, 1, 0, 0, 1, 0, 0, 1, 0, 1, 1, 1, 1, 0, 1]
20 20
Alice sifted key:  [0, 1, 1, 0, 0, 1, 0, 0, 1, 1]
Bob sifted key:  [0, 1, 1, 0, 0, 1, 0, 0, 1, 1]
5
[0, 1, 1, 0, 0]
Success rate:  1.0
No eavesdropper detected.
Alice key:  [1, 0, 0, 1, 1]
Bob key:  [1, 0, 0, 1, 1]
